# Annotate RegionDS

Following DMR calling (or any other RegionDS created from other DMR and genomic region sets), we can annotate the DMR regions by addtition genomic annotations stored in BigWig or BED format.


## Import

In [2]:
import pandas as pd
import pathlib
from ALLCools.mcds import RegionDS

## Open RegionDS

In [5]:
dmr_ds = RegionDS.open('HIP_small')
dmr_ds

Using dmr as region_dim


<xarray.RegionDS>
Dimensions:                 (count_type: 2, dmr: 131, sample: 20, sample_collapsed: 10, snATAC: 10, genome-features: 29, TE: 32, motif: 2179, motif_value: 3, motif-cluster: 286)
Coordinates: (12/15)
  * count_type              (count_type) <U3 'mc' 'cov'
  * dmr                     (dmr) <U9 'chr1-0' 'chr1-1' ... 'chr19-122'
    dmr_chrom               (dmr) <U5 'chr1' 'chr1' 'chr1' ... 'chr19' 'chr19'
    dmr_end                 (dmr) int64 10002172 10003542 ... 5099203 5099952
    dmr_length              (dmr) int64 2 305 54 2 2 2 ... 632 842 195 399 335
    dmr_ndms                (dmr) int64 1 7 2 1 1 1 2 13 ... 7 13 19 9 9 3 6 13
    ...                      ...
  * snATAC                  (snATAC) <U4 'CA23' 'CGE' 'ASC' ... 'OPC' 'DG'
  * genome-features         (genome-features) <U31 'CGI_promoter.all' ... 'bl...
  * TE                      (TE) <U17 'DNA.DNA' 'DNA.MULE-MuDR' ... 'SINE.tRNA'
  * motif                   (motif) <U29 'ALX3_homeodomain_2' ... 'ZSC31_HUMA...
  * motif_value             (motif_value) <U11 'n_motifs' ... 'total_score'
  * motif-cluster           (motif-cluster) object 'c1' 'c10' ... 'c98' 'c99'
Data variables:
    dmr_da                  (sample, dmr, count_type) uint32 ...
    dmr_da_frac             (sample, dmr) float32 ...
    dmr_state               (sample, dmr) int8 ...
    dmr_state_collapsed     (dmr, sample_collapsed) int8 ...
    dmr_snATAC_da           (dmr, snATAC) float32 ...
    dmr_genome-features_da  (dmr, genome-features) bool ...
    dmr_TE_da               (dmr, TE) bool ...
    dmr_motif_da            (dmr, motif, motif_value) uint16 ...
    dmr_motif-cluster_da    (motif-cluster, dmr, motif_value) uint16 ...
Attributes:
    chrom_size_path:     /home/hanliu/project/allcools_doc/cluster_level/Regi...
    region_dim:          dmr
    region_ds_location:  /home/hanliu/project/allcools_doc/cluster_level/Regi...

In [7]:
RegionDS.open('HIP_small/dms/', region_dim='dms')

<xarray.RegionDS>
Dimensions:       (count_type: 2, dms: 767, sample: 20)
Coordinates:
  * count_type    (count_type) <U3 'mc' 'cov'
  * dms           (dms) <U13 'chr1-10000325' 'chr1-10001697' ... 'chr19-5099951'
    dms_chrom     (dms) <U5 'chr1' 'chr1' 'chr1' ... 'chr19' 'chr19' 'chr19'
    dms_contexts  (dms) <U3 'CGG' 'CGC' 'CGG' 'CGA' ... 'CGG' 'CGA' 'CGC' 'CGA'
    dms_p-values  (dms) float64 0.001667 0.008667 ... 0.0003333 0.0003333
    dms_pos       (dms) int64 10000325 10001697 10002171 ... 5099924 5099951
  * sample        (sample) <U18 'snm3C_ASC' 'snm3C_CA1' ... 'snmC_OPC'
Data variables:
    dms_da        (sample, dms, count_type) float64 ...
    dms_da_frac   (sample, dms) float64 ...
    dms_residual  (sample, dms) float64 ...
Attributes:
    region_dim:  dms

## DMR Chromatin Accessibility Profile (BigWig)

For example, here we annotate cluster-matched chromatin accessibility profiles from a mouse hippocampus snATAC-seq dataset. Each profile is stored in BigWig format.

In [3]:
# prepare the bigwig tab-separated table, first column is cluster name, second column is BigWig path
bigwig_dir = '/gale/netapp/cemba3c/projects/ALLCools/HIPBulk/atac_bulk/'
bigwigs = pd.Series({
    p.name.split('.')[0].split('_')[-1]: str(p)
    for p in pathlib.Path(bigwig_dir).glob('HIP_snATAC_*.bw')
})
bigwigs.to_csv('bigwig.csv', header=False)
bigwigs

CA23    /gale/netapp/cemba3c/projects/ALLCools/HIPBulk...
CGE     /gale/netapp/cemba3c/projects/ALLCools/HIPBulk...
ASC     /gale/netapp/cemba3c/projects/ALLCools/HIPBulk...
MGE     /gale/netapp/cemba3c/projects/ALLCools/HIPBulk...
CA1     /gale/netapp/cemba3c/projects/ALLCools/HIPBulk...
ODC     /gale/netapp/cemba3c/projects/ALLCools/HIPBulk...
MGC     /gale/netapp/cemba3c/projects/ALLCools/HIPBulk...
NonN    /gale/netapp/cemba3c/projects/ALLCools/HIPBulk...
OPC     /gale/netapp/cemba3c/projects/ALLCools/HIPBulk...
DG      /gale/netapp/cemba3c/projects/ALLCools/HIPBulk...
dtype: object

In [4]:
dmr_ds.annotate_by_bigwigs(slop=250,
                           bigwig_table='bigwig.csv',
                           dim='snATAC',
                           cpu=30)

Use chunk size 2


## DMR Overlapping Genome Features (BED)

Next, we overlap the DMR regions with a set of BED files that containing different kinds of genome features. The output dataset is a boolean matrix descriping whether a DMR is overlapping one feature kind (e.g. CGI).

In [5]:
genome_feature_dir = '/home/hanliu/ref/mouse/genome_feature/'
genome_feature_beds = {
    p.name[:-4]: str(p)
    for p in pathlib.Path(genome_feature_dir).glob('*.bed')
}

# also annotate blacklist to filter DMR
genome_feature_beds['blacklist'] = '/home/hanliu/ref/blacklist/mm10-blacklist.v2.bed.gz'

beds = pd.Series(genome_feature_beds)
beds.to_csv('genome_featue_bed.csv', header=False)
beds

CGI_promoter.all                   /home/hanliu/ref/mouse/genome_feature/CGI_prom...
CGI_promoter.protein_coding        /home/hanliu/ref/mouse/genome_feature/CGI_prom...
exon.all                           /home/hanliu/ref/mouse/genome_feature/exon.all...
exon.first                         /home/hanliu/ref/mouse/genome_feature/exon.fir...
exon.protein_coding                /home/hanliu/ref/mouse/genome_feature/exon.pro...
gene.all                           /home/hanliu/ref/mouse/genome_feature/gene.all...
gene.lincRNA                       /home/hanliu/ref/mouse/genome_feature/gene.lin...
gene.protein_coding                /home/hanliu/ref/mouse/genome_feature/gene.pro...
intron.all                         /home/hanliu/ref/mouse/genome_feature/intron.a...
intron.first                       /home/hanliu/ref/mouse/genome_feature/intron.f...
intron.protein_coding              /home/hanliu/ref/mouse/genome_feature/intron.p...
Non_CGI_promoter.all               /home/hanliu/ref/mouse/genome_

In [6]:
dmr_ds.annotate_by_bed(slop=250,
                       bed_table='genome_featue_bed.csv',
                       dim='genome-features',
                       bed_sorted=False,
                       cpu=30)

Use chunk size 2


## DMR Overlapping TE (BED)

Finally, we overlap the DMR regions with Transposable Elements (TE) collected from Repeat Master mouse genome. The output dataset is a boolean matrix descriping whether a DMR is overlapping one TE kind.

In [7]:
bed_dir = '/home/hanliu/ref/mouse/ucsc/TE_Beds/'
beds = {
    p.name[:-4]: str(p)
    for p in pathlib.Path(bed_dir).glob('*.bed')
}
beds = pd.Series(beds)
beds.to_csv('te_bed.csv', header=False)
beds

DNA.DNA                /home/hanliu/ref/mouse/ucsc/TE_Beds/DNA.DNA.bed
DNA.MULE-MuDR        /home/hanliu/ref/mouse/ucsc/TE_Beds/DNA.MULE-M...
DNA.MuDR              /home/hanliu/ref/mouse/ucsc/TE_Beds/DNA.MuDR.bed
DNA.PiggyBac         /home/hanliu/ref/mouse/ucsc/TE_Beds/DNA.PiggyB...
DNA.TcMar            /home/hanliu/ref/mouse/ucsc/TE_Beds/DNA.TcMar.bed
DNA.TcMar-Mariner    /home/hanliu/ref/mouse/ucsc/TE_Beds/DNA.TcMar-...
DNA.TcMar-Pogo       /home/hanliu/ref/mouse/ucsc/TE_Beds/DNA.TcMar-...
DNA.TcMar-Tc2        /home/hanliu/ref/mouse/ucsc/TE_Beds/DNA.TcMar-...
DNA.TcMar-Tigger     /home/hanliu/ref/mouse/ucsc/TE_Beds/DNA.TcMar-...
DNA.hAT                /home/hanliu/ref/mouse/ucsc/TE_Beds/DNA.hAT.bed
DNA.hAT-Blackjack    /home/hanliu/ref/mouse/ucsc/TE_Beds/DNA.hAT-Bl...
DNA.hAT-Charlie      /home/hanliu/ref/mouse/ucsc/TE_Beds/DNA.hAT-Ch...
DNA.hAT-Tip100       /home/hanliu/ref/mouse/ucsc/TE_Beds/DNA.hAT-Ti...
LINE.CR1              /home/hanliu/ref/mouse/ucsc/TE_Beds/LINE.CR1.bed
LINE.D

In [8]:
dmr_ds.annotate_by_bed(slop=250,
                       bed_table='te_bed.csv',
                       dim='TE',
                       bed_sorted=False,
                       cpu=30)

Use chunk size 2


## After Annotation
After annotation, the RegionDS will contain additional dmr-by-feature matrix that can be used for following analysis.

In [9]:
dmr_ds

<xarray.RegionDS>
Dimensions:                 (count_type: 2, dmr: 131, sample: 20, snATAC: 10, genome-features: 29, TE: 32)
Coordinates:
  * count_type              (count_type) <U3 'mc' 'cov'
  * dmr                     (dmr) <U9 'chr1-0' 'chr1-1' ... 'chr19-122'
    dmr_chrom               (dmr) <U5 'chr1' 'chr1' 'chr1' ... 'chr19' 'chr19'
    dmr_end                 (dmr) int64 10002172 10003542 ... 5099203 5099952
    dmr_length              (dmr) int64 2 305 54 2 2 2 ... 632 842 195 399 335
    dmr_ndms                (dmr) int64 1 7 2 1 1 1 2 13 ... 7 13 19 9 9 3 6 13
    dmr_start               (dmr) int64 10002170 10003237 ... 5098804 5099617
  * sample                  (sample) <U18 'snm3C_ASC' 'snm3C_CA1' ... 'snmC_OPC'
  * snATAC                  (snATAC) <U4 'CA23' 'CGE' 'ASC' ... 'OPC' 'DG'
  * genome-features         (genome-features) <U31 'CGI_promoter.all' ... 'bl...
  * TE                      (TE) <U17 'DNA.DNA' 'DNA.MULE-MuDR' ... 'SINE.tRNA'
Data variables:
    dmr_da                  (sample, dmr, count_type) uint32 35 65 ... 55 188
    dmr_da_frac             (sample, dmr) float32 0.5385 0.8161 ... 0.2926
    dmr_state               (sample, dmr) int8 -1 -1 -1 1 -1 -1 ... 1 1 1 0 0 -1
    dmr_snATAC_da           (dmr, snATAC) float32 dask.array<chunksize=(131, 2), meta=np.ndarray>
    dmr_genome-features_da  (dmr, genome-features) bool dask.array<chunksize=(131, 2), meta=np.ndarray>
    dmr_TE_da               (dmr, TE) bool dask.array<chunksize=(131, 2), meta=np.ndarray>
Attributes:
    region_dim:          dmr
    region_ds_location:  /home/hanliu/project/allcools_doc/cluster_level/Regi...
    chrom_size_path:     /home/hanliu/project/allcools_doc/cluster_level/Regi...

### Selectively open
You may also select specific datasets to open for future analysis.

In [10]:
RegionDS.open('HIP_small', select_dir=['dmr', 'dmr_TE'])

Using dmr as region_dim


<xarray.RegionDS>
Dimensions:      (count_type: 2, dmr: 131, sample: 20, TE: 32)
Coordinates:
  * count_type   (count_type) <U3 'mc' 'cov'
  * dmr          (dmr) <U9 'chr1-0' 'chr1-1' ... 'chr19-121' 'chr19-122'
    dmr_chrom    (dmr) <U5 'chr1' 'chr1' 'chr1' ... 'chr19' 'chr19' 'chr19'
    dmr_end      (dmr) int64 10002172 10003542 10003967 ... 5099203 5099952
    dmr_length   (dmr) int64 2 305 54 2 2 2 10 ... 589 924 632 842 195 399 335
    dmr_ndms     (dmr) int64 1 7 2 1 1 1 2 13 3 2 1 ... 2 1 2 7 13 19 9 9 3 6 13
    dmr_start    (dmr) int64 10002170 10003237 10003913 ... 5098804 5099617
  * sample       (sample) <U18 'snm3C_ASC' 'snm3C_CA1' ... 'snmC_ODC' 'snmC_OPC'
  * TE           (TE) <U17 'DNA.DNA' 'DNA.MULE-MuDR' ... 'SINE.MIR' 'SINE.tRNA'
Data variables:
    dmr_da       (sample, dmr, count_type) uint32 ...
    dmr_da_frac  (sample, dmr) float32 ...
    dmr_state    (sample, dmr) int8 ...
    dmr_TE_da    (dmr, TE) bool ...
Attributes:
    region_dim:          dmr
    region_ds_location:  /home/hanliu/project/allcools_doc/cluster_level/Regi...
    chrom_size_path:     /home/hanliu/project/allcools_doc/cluster_level/Regi...